# 4.2 머신 러닝 모델 평가
- 머신 러닝의 목표 : 처음 본 데이터에서 잘 작동하는 **일반화**된 모델을 얻는 것
    - **과대적합**이 주요 장애물

## 4.2.1 훈련, 검증, 테스트 세트
- 모델 평가의 핵심은 항상 훈련, 검증, 테스트 3개의 세트로 나누는 것
- 층의 수나 유닛 수 등 **하이퍼파라미터**(hyperparameter)를 선택하기 위해 모델의 설정을 튜닝
- 검증 세트의 성능을 기반으로 모델의 설정을 튜닝하면 검증 세트에 과대적합
- **정보 누설**(information leak) : 검증 세트의 모델 성능에 기반하여 모델의 하이퍼파라미터를 조절할 때마다 검증 데이터에 관한 정보가 모델로 새는 것

### 단순 홀드아웃 검증
1. 데이터의 일정량을 테스트 세트로 떼어 놓음
2. 훈련 데이터에서 검증 세트를 떼어 놓음

### 홀드아웃 검증 구현 예
- 단점 : 데이터가 적을 때 검증 세트와 테스트 세트의 샘플이 너무 적어 주어진 전체 데이터를 통계적으로 대표하지 못할 수 있음

```python
num_validation_samples = 10000

np.random.shuffle(data) # 데이터를 섞는 것이 좋음

validation_data = data[:num_validation_samples] # 검증 세트
data = data[num_validation_samples:]

training_data = data[:] # 훈련 세트

model = get_model()
model.train(training_data) # 훈련 세트에서 모델을 훈련
validation_score = model.evaluate(validation_data) # 검증 세트로 평가

# 튜닝, 훈련, 평가 반복

model = get_model()
model.train(np.concatenate([training_data, validation_data])) # 하이퍼파라미터 튜닝이 끝나면 테스트 데이터를 제외한 모든 데이터를 사용하여 모델을 다시 훈련
test_score = model.evaluate(test_data)
```


### K-겹 교차 검증
1. 데이터를 동일한 크기를 가진 K개 분할로 나눔
2. 각 분할 i에 대해 남은 K-1개의 분할로 모델을 훈련하고 분할 i에서 모델을 평가
- 모델의 성능이 데이터 분할에 따라 편차가 클 때 도움이 됨

### K-겹 교차 검증 구현 예
- 사이킷런의 `cross_validate()` 함수를 사용하여 구현 가능
    - 사이킷런과 호환되도록 `KerasClassifier`, `KerasRegressor` 클래스로 모델을 감싸야함

```python
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    validation_data = data[num_validation_samples * fold: num_validation_samples * (fold + 1)] # 검증 세트
    training_data = data[:num_validation_samples * fold] + data[num_validation_samples * (fold + 1):] # 훈련 세트

    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)

validation_score = np.average(validation_scores) # 검증 점수: K개 폴드의 검증 점수 평균

# 테스트 데이터를 제외한 전체 데이터로 최종 모델 훈련
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)
```

### 셔플링을 사용한 반복 K-겹 교차 검증
- 비교적 가용 데이터가 적고 가능한 정확하게 모델을 평가하고자 할 때 사용
- K-겹 교차 검증을 여러 번 적용하되 K개로 분할하기 전에 매번 데이터를 무작위로 섞음
- 최종 점수는 모든 K-겹 교차 검증을 실행해서 얻은 점수의 평균
- P X K개의 모델을 훈련, 평가하므로 비용이 매우 많이 든다는 단점이 있음
- 사이킷런의 `RepeatedKFlod`(회귀)와 `RepeatedStratifiedKFold`(분류) 클래스를 `cross_validate()` 함수에 적용하여 구현 가능

## 4.2.2 기억해야 할 것
- **대표성 있는 데이터** : 훈련 세트와 테스트 세트가 주어진 데이터에 대한 대표성이 있어야 함
    - 사이킷런의 `train_test_split()` 함수의 `stratify` 매개변수로 계층별(stratified) 분할을 수행
- **시간의 방향** : 과거로부터 미래를 예측하려 한다면 데이터를 무작위로 섞어서는 안 됨
    - 훈련 세트의 데이터보다 테스트 세트의 데이터가 더 미래의 것이어야 함
- **데이터 중복** : 한 데이터셋에 어떤 데이터가 두 번 등장하며 훈련과 테스트로 나누어진다면, 훈련 데이터의 일부로 테스트를 하는 최악의 경우가 됨